In [11]:
import os
import time
import polars as pl
from polars import Config


fpath1 = "C:\\data_tests\\data\\kyrgyz_aws.csv"
files = {
    "Site_A": fpath1,
    
}



# **処理時間の計測**
start_time = time.time()

# dfを格納するdict
dfs = {}

for site_name, file in files.items():
    # awsデータの読み込み
    df = (
        pl.scan_csv(file, encoding="utf8-lossy", skip_rows=1)
        .filter(~pl.col("TIMESTAMP").is_null())
        .collect()
    )

    # 単位の行を取得してdictに格納
    unit_row = df.row(0)
    unit_dict = {col: unit_row[i] for i, col in enumerate(df.columns)}
    print(unit_dict)

    # 単位の行を削除
    df = df.slice(1)

    # バッテリーが12V未満のデータはログ出力して分離
    df_below_12v = df.filter(df["BattVolt_10min_Min"] < 12)
    df = df.filter(df["BattVolt_10min_Min"] >= 12)
    print(df_below_12v)

    # RECORD列を地点情報に書き換える
    df = df.with_columns(pl.lit(site_name).alias("Location"))


    # TIMESTAMPをUNIXTIMEに変換
    df = df.with_columns(
        pl.col("TIMESTAMP").str.to_datetime("%Y/%m/%d %H:%M", strict=False).dt.epoch()
    )
    # 変換出来なかった行をログ出力
    df_null_unixtime = df.filter(pl.col("TIMESTAMP").is_null())
    print(df_null_unixtime)

    # dictに保存
    dfs[site_name] = df

    # **計測終了**
    end_time = time.time()
    print(f"🔥 scan_csv() 実行時間: {end_time - start_time:.4f} 秒")


df

{'TIMESTAMP': 'TS', 'RECORD': 'RN', 'WS_10min_Avg': 'm/s', 'WD_10min_Avg': 'Deg', 'WS_10min_Max': 'm/s', 'AirTemp_10min_Avg': 'DegC', 'AirRH_10min_Avg': '%', 'Atm_10min_Avg': 'hPa', 'Rain_10min_Tot': 'mm', 'SR_dn_10min_Avg': 'W/m2', 'SR_up_10min_Avg': 'W/m2', 'IR_dn_10min_Avg': 'W/m2', 'IR_up_10min_Avg': 'W/m2', 'IR_up_Temp_10min_Avg': 'DegC', 'IR_dn_Temp_10min_Avg': 'DegC', 'NetRs_10min_Avg': 'W/m2', 'NetRl_10min_Avg': 'W/m2', 'Albedo_10_min_Avg': None, 'NetTot_10min_Avg': 'W/m2', 'IR_dn_Corr_10min_Avg': 'W/m2', 'IR_up_Corr_10min_Avg': 'W/m2', 'BattVolt_10min_Min': 'V'}
shape: (2, 22)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ TIMESTAMP  ┆ RECORD ┆ WS_10min_A ┆ WD_10min_ ┆ … ┆ NetTot_10 ┆ IR_dn_Cor ┆ IR_up_Cor ┆ BattVolt_ │
│ ---        ┆ ---    ┆ vg         ┆ Avg       ┆   ┆ min_Avg   ┆ r_10min_A ┆ r_10min_A ┆ 10min_Min │
│ str        ┆ str    ┆ ---        ┆ ---       ┆   ┆ ---       ┆ vg        ┆ vg        ┆ ---       │
│  

TIMESTAMP,RECORD,WS_10min_Avg,WD_10min_Avg,WS_10min_Max,AirTemp_10min_Avg,AirRH_10min_Avg,Atm_10min_Avg,Rain_10min_Tot,SR_dn_10min_Avg,SR_up_10min_Avg,IR_dn_10min_Avg,IR_up_10min_Avg,IR_up_Temp_10min_Avg,IR_dn_Temp_10min_Avg,NetRs_10min_Avg,NetRl_10min_Avg,Albedo_10_min_Avg,NetTot_10min_Avg,IR_dn_Corr_10min_Avg,IR_up_Corr_10min_Avg,BattVolt_10min_Min,Location
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1664197800000000,"""2""","""1.808""","""258.4""","""2.7""","""5.836""","""31.73""","""646.416""","""0""","""928.9788""","""126.6331""","""-126.2469""","""32.10106""","""12.11""","""11.59""","""802.3458""","""-158.348""","""0.1363156""","""643.9978""","""249.1938""","""404.7926""","""13.56""","""Site_A"""
1664198400000000,"""3""","""2.535""","""265.2""","""3.4""","""5.668""","""30.55""","""646.3317""","""0""","""924.3484""","""125.8074""","""-126.3353""","""33.52101""","""11.73""","""11.24""","""798.541""","""-159.8563""","""0.1361013""","""638.6846""","""247.1382""","""404.3836""","""13.56""","""Site_A"""
1664199000000000,"""4""","""1.972""","""269.5""","""2.9""","""5.533""","""30.79""","""646.2999""","""0""","""342.7564""","""123.825""","""-41.3195""","""36.95861""","""10.64""","""10.4""","""218.9314""","""-78.27811""","""9.071039""","""140.6533""","""326.4338""","""403.498""","""13.57""","""Site_A"""
1664199600000000,"""5""","""1.473""","""268.7""","""3.3""","""6.115""","""27.35""","""646.2983""","""0""","""9.188166""","""134.3892""","""-0.02174615""","""41.87482""","""9.85""","""9.89""","""-125.201""","""-41.89656""","""22.7164""","""-167.0976""","""363.6657""","""405.7614""","""13.59""","""Site_A"""
1664200200000000,"""6""","""2.797""","""267.8""","""6.3""","""6.163""","""33.04""","""646.2183""","""0""","""647.0114""","""130.6579""","""-84.65701""","""44.71037""","""9.79""","""9.53""","""516.3536""","""-129.3674""","""4.652904""","""386.9862""","""278.6996""","""406.7659""","""13.59""","""Site_A"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1720862400000000,"""94443""","""5.7""","""99""","""5.7""","""3.7""","""74.4""","""640.2999""","""0""","""759.8738""","""85.99303""","""-61.88894""","""6.373354""","""7.585""","""7.03""","""673.8807""","""-68.26229""","""0.1131257""","""605.6184""","""290.2963""","""355.7825""","""13.56""","""Site_A"""
1720863000000000,"""94444""","""5.7""","""99""","""5.7""","""3.7""","""74.4""","""640.2999""","""0""","""710.0383""","""81.92848""","""-66.81017""","""8.499489""","""8.17""","""7.669""","""628.1098""","""-75.30966""","""0.1149435""","""552.8002""","""288.3352""","""361.1105""","""13.56""","""Site_A"""
1720863600000000,"""94445""","""5.7""","""99""","""5.7""","""3.7""","""74.4""","""640.2999""","""0""","""676.0035""","""75.55333""","""-63.78199""","""8.74075""","""9.3""","""8.78""","""600.4501""","""-72.52273""","""0.1101413""","""527.9274""","""297.0933""","""366.9775""","""13.56""","""Site_A"""
